## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive
%cd [2024-2025] AN2DL Homework 2

Mounted at /gdrive
/gdrive/My Drive
/gdrive/My Drive/[2024-2025] AN2DL Homework 2


## ⚙️ Import Libraries

In [2]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import mode

import matplotlib.pyplot as plt
%matplotlib inline

seed = 29
np.random.seed(seed)
tf.random.set_seed(seed)

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

## ⏳ Load the Datasets

In [74]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)
Input shape: (64, 128, 1)
Number of classes: 5


## ❌ Remove outliers from dataset

In [75]:
# Lists to contain filtered elements
X_train_filtered = []
y_train_filtered = []

for i in range(len(y_train)):
    label = y_train[i].argmax() if y_train.ndim > 1 else y_train[i]
    if label != 415:
        # Add to filtered dataset the non-alien images
        X_train_filtered.append(X_train[i])
        y_train_filtered.append(y_train[i])

# Convert lists to numpy arrays
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

print(f"Shape X_train_filtered: {X_train_filtered.shape}")
print(f"Shape y_train_filtered: {y_train_filtered.shape}")
print(f"Unique classes: {np.unique(y_train_filtered)}")

Shape X_train_filtered: (2505, 64, 128, 1)
Shape y_train_filtered: (2505, 64, 128)
Unique classes: [0. 1. 2. 3. 4.]


## ✂ Split into Training and Validation Sets

In [76]:
# Split the training dataset to get a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_filtered,
    y_train_filtered,
    test_size=0.1,
    random_state=seed)

# Print the shapes of the resulting sets
print('Training set shape:\t', X_train.shape, y_train.shape)
print('Validation set shape:\t', X_val.shape, y_val.shape)

Training set shape:	 (2254, 64, 128, 1) (2254, 64, 128)
Validation set shape:	 (251, 64, 128, 1) (251, 64, 128)


## 🧮 Define network parameters

In [77]:
# Set batch size for training
batch_size = 64

# Set learning rate for the optimiser
learning_rate = 1e-5

# Set early stopping patience threshold
patience = 10

# Set maximum number of training epochs
epochs = 100

# Set data split size for training and validation
split_size = 300

from sklearn.utils.class_weight import compute_class_weight

# Assumi che `y_train` sia l'array delle etichette nel tuo dataset di allenamento
# Se `y_train` è un array 3D (ad esempio immagini con etichette pixel-wise), lo appiattiamo prima:
y_train_flat = y_train.flatten()

# Calcola i pesi delle classi
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_flat),
    y=y_train_flat
)

# Converte i pesi in un dizionario
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}
class_weights[0] = 0.0

print("Class Weights:", class_weights)


Class Weights: {0: 0.0, 1: 0.5910939761597104, 2: 0.8421959857784994, 3: 1.1219193842338118, 4: 153.08214226496435}


In [78]:
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_mean_iou',
    patience=10,
    restore_best_weights=True,
    mode='max'
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_mean_iou",  # Cambia con il nome corretto della tua metrica
    factor=0.5,  # Riduci il learning rate del 50%
    patience=5,  # Numero di epoch senza miglioramento
    min_lr=1e-6  # Limite minimo per il learning rate
)



# Store the callback in a list
callbacks = [early_stopping, lr_scheduler]

##Augmentation


In [79]:
def augment_data(image, label):
    # Geometric Transformations
    image = tf.image.random_flip_left_right(image)
    label = tf.image.random_flip_left_right(label)

    image = tf.image.random_flip_up_down(image)
    label = tf.image.random_flip_up_down(label)

    # Chromatic Transformations
    #image = tf.image.random_brightness(image, max_delta=0.4)
    #image = tf.image.random_contrast(image, lower=0.7, upper=1.3)

    return image, label

def preprocess_image(image):
    image = tf.expand_dims(image, axis=-1) if len(image.shape) == 2 else image
    image = tf.cast(image, tf.float32)
    return image

def preprocess_label(label):
    label = tf.expand_dims(label, axis=-1) if len(label.shape) == 2 else label
    label = tf.cast(label, tf.int32)
    return label

def preprocess_data(image, label):
    image = preprocess_image(image)
    label = preprocess_label(label)
    return image, label

In [80]:
# Dataset preprocessato (senza augmentation)
train_dataset_static = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset_static = train_dataset_static.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset_static = train_dataset_static.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Dataset con augmentation
train_dataset_dynamic = train_dataset_static.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)
train_dataset_dynamic2 = train_dataset_static.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)
#train_dataset_dynamic3 = train_dataset_static.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)


# Dataset combinato
combined_dataset = train_dataset_static.concatenate(train_dataset_dynamic)
combined_dataset = combined_dataset.concatenate(train_dataset_dynamic2)
#combined_dataset = combined_dataset.concatenate(train_dataset_dynamic3)
combined_dataset = combined_dataset.shuffle(buffer_size=len(X_train))
combined_dataset = combined_dataset.prefetch(tf.data.AUTOTUNE)

# Validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

## ⭐ Define Loss function

In [81]:
def jaccard_loss_no_background(y_true, y_pred, smooth=1e-6):
    """
    Jaccard Loss per segmentazione semantica che ignora la classe 0 (background).

    Args:
        y_true: Ground truth, con valori interi per ogni pixel.
        y_pred: Predizioni del modello, come probabilità per ogni classe.
        smooth: Termine per evitare divisioni per zero.

    Returns:
        Valore della Jaccard Loss.
    """
    # Converti y_true in one-hot encoding e rimuovi dimensioni extra
    y_true = tf.squeeze(y_true, axis=-1)  # Rimuovi dimensione extra se presente
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])

    # Escludi la classe 0 (background)
    y_true = y_true[..., 1:]  # Ignora la classe 0
    y_pred = y_pred[..., 1:]  # Ignora la classe 0

    # Calcola intersezione e unione
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2]) - intersection

    # Calcola IoU (Intersection over Union)
    iou = (intersection + smooth) / (union + smooth)

    # Jaccard Loss
    return 1 - tf.reduce_mean(iou)

In [82]:
def dice_loss_no_background(y_true, y_pred, smooth=1e-6):
    """
    Dice Loss per segmentazione semantica che ignora la classe 0 (background).

    Args:
        y_true: Ground truth, con valori interi per ogni pixel.
        y_pred: Predizioni del modello, come probabilità per ogni classe.
        smooth: Termine per evitare divisioni per zero.

    Returns:
        Valore della Dice Loss.
    """
    # Converti y_true in one-hot encoding e rimuovi dimensioni extra
    y_true = tf.squeeze(y_true, axis=-1)  # Rimuovi dimensione extra se presente
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])

    # Escludi la classe 0 (background)
    y_true = y_true[..., 1:]  # Ignora la classe 0
    y_pred = y_pred[..., 1:]  # Ignora la classe 0

    # Calcola l'intersezione e la somma
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    total = tf.reduce_sum(y_true + y_pred, axis=[1, 2])

    # Calcola il coefficiente Dice
    dice_coeff = (2 * intersection + smooth) / (total + smooth)

    # Dice Loss
    return 1 - tf.reduce_mean(dice_coeff)


In [83]:
def focal_loss_no_background(y_true, y_pred, gamma=2.0, alpha=0.25):
    """
    Focal Loss per segmentazione semantica che ignora la classe 0 (background).

    Args:
        y_true: Ground truth, con valori interi per ogni pixel.
        y_pred: Predizioni del modello, come probabilità per ogni classe.
        gamma: Fattore di messa a fuoco per penalizzare previsioni errate.
        alpha: Ponderazione della perdita.

    Returns:
        Valore della Focal Loss.
    """
    # Converti y_true in one-hot encoding e rimuovi dimensioni extra
    y_true = tf.squeeze(y_true, axis=-1)
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])

    # Escludi la classe 0 (background)
    y_true = y_true[..., 1:]
    y_pred = y_pred[..., 1:]

    # Evita log(0) con un termine epsilon
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

    # Calcola focal loss
    cross_entropy = -y_true * tf.math.log(y_pred)
    weight = alpha * tf.math.pow(1 - y_pred, gamma)
    loss = weight * cross_entropy

    return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))


In [84]:
def combined_loss_no_background(y_true, y_pred, jaccard_weight=0.5, dice_weight=0.3, focal_weight=0.2):
    """
    Combined Loss che unisce Jaccard, Dice e Focal Loss ignorando la classe 0 (background).

    Args:
        y_true: Ground truth, con valori interi per ogni pixel.
        y_pred: Predizioni del modello, come probabilità per ogni classe.
        jaccard_weight: Ponderazione della Jaccard Loss.
        dice_weight: Ponderazione della Dice Loss.
        focal_weight: Ponderazione della Focal Loss.

    Returns:
        Valore della Combined Loss.
    """
    jaccard = jaccard_loss_no_background(y_true, y_pred)
    dice = dice_loss_no_background(y_true, y_pred)
    #focal = focal_loss_no_background(y_true, y_pred)

    return 0.5 * jaccard + 0.5 * dice


## 🔨 Build the model

In [85]:
def res_unet_block(input_tensor, filters, name):
    x = tfkl.Conv2D(filters, (3, 3), padding="same", kernel_initializer="he_normal", name=f"{name}_conv1")(input_tensor)
    x = tfkl.BatchNormalization(name=f"{name}_bn1")(x)
    x = tfkl.Activation("relu", name=f"{name}_act1")(x)
    x = tfkl.Conv2D(filters, (3, 3), padding="same", kernel_initializer="he_normal", name=f"{name}_conv2")(x)
    x = tfkl.BatchNormalization(name=f"{name}_bn2")(x)

    # Skip connection
    shortcut = tfkl.Conv2D(filters, (1, 1), padding="same", name=f"{name}_shortcut")(input_tensor)
    x = tfkl.Add(name=f"{name}_add")([x, shortcut])
    x = tfkl.Activation("relu", name=f"{name}_act2")(x)
    return x

# Usa il blocco ResNet-like nei livelli della U-Net


In [86]:
def adaptive_dropout(rate, epoch):
    return max(0.5 * (1 - epoch / 100), 0.1)  # Dropout diminuisce da 0.5 a 0.1

class AdaptiveDropout(tfkl.Layer):
    def __init__(self, initial_rate):
        super(AdaptiveDropout, self).__init__()
        self.rate = initial_rate

    def call(self, inputs, training=None):
        if training:
            rate = adaptive_dropout(self.rate, tf.keras.backend.get_value(self.add_weight('epoch', initializer='zeros')))
            return tf.nn.dropout(inputs, rate=rate)
        return inputs


In [87]:
class InstanceNormalization(tfk.layers.Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        # We learn the scale (gamma) and offset (beta) parameters for the normalization
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:], initializer='ones', trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:], initializer='zeros', trainable=True)
        super(InstanceNormalization, self).build(input_shape)

    def call(self, inputs):
        # Calculate the mean and variance across the spatial dimensions (height and width)
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        normalized = (inputs - mean) / tf.sqrt(variance + self.epsilon)
        return self.gamma * normalized + self.beta


In [88]:
def attention_gate(x, g, filters):
    # Theta_x: convoluzione sul tensor x (feature map dal downsampling)
    theta_x = tfkl.Conv2D(filters, (1, 1), strides=(1, 1), padding="same")(x)
    # Phi_g: convoluzione sul tensor g (output del layer precedente nel decoder)
    phi_g = tfkl.Conv2D(filters, (1, 1), strides=(1, 1), padding="same")(g)
    # Addizione dei due risultati convoluzionati
    f = tfkl.Add()([theta_x, phi_g])
    f = tfkl.Activation('relu')(f)
    # Psi: convoluzione finale per ottenere la maschera di attenzione
    psi = tfkl.Conv2D(1, (1, 1), strides=(1, 1), padding="same", activation='sigmoid')(f)
    # Moltiplica x per la maschera di attenzione
    return tfkl.Multiply()([x, psi])

In [102]:
# Define the input layer
input_layer = tfkl.Input(shape=input_shape)


# Downsampling path with an extra layer
def downsampling_block(input_tensor, filters, name):
    x = res_unet_block(input_tensor, filters, name=f"{name}_unet")
    # Extra convolutional layer
    x = tfkl.Conv2D(filters, (3, 3), padding="same", kernel_initializer="he_normal", name=f"{name}_extra_conv")(x)
    x = tfkl.BatchNormalization(name=f"{name}_extra_bn")(x)
    x = tfkl.Activation("relu", name=f"{name}_extra_act")(x)
    pool = tfkl.MaxPooling2D(pool_size=(2, 2), name=f"{name}_pool")(x)
    return x, pool


down_block_1, d1 = downsampling_block(input_layer, 64, name='down_block1')
down_block_2, d2 = downsampling_block(d1, 128, name='down_block2')
down_block_3, d3 = downsampling_block(d2, 256, name='down_block3')
down_block_4, d4 = downsampling_block(d3, 512, name='down_block4')

# Bottleneck with multi-scale features and Squeeze-and-Excitation
def bottleneck_module(input_tensor, filters):
    # Multi-scale convolutions
    conv1 = tfkl.Conv2D(filters, kernel_size=3, padding='same', activation='relu', name='bottleneck_conv1')(input_tensor)
    conv3 = tfkl.Conv2D(filters, kernel_size=5, padding='same', activation='relu', name='bottleneck_conv3')(input_tensor)
    conv_dilated = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=2, padding='same', activation='relu', name='bottleneck_dilated')(input_tensor)

    # Concatenate features
    concat = tfkl.Concatenate(name='bottleneck_concat')([conv1, conv3, conv_dilated])
    concat = tfkl.Conv2D(filters, kernel_size=1, activation='relu', name='bottleneck_output')(concat)

    # Squeeze-and-Excitation Block
    se = tfkl.GlobalAveragePooling2D()(concat)
    se = tfkl.Dense(filters // 16, activation='relu', name='se_reduce')(se)
    se = tfkl.Dense(filters, activation='sigmoid', name='se_expand')(se)
    se = tfkl.Reshape((1, 1, filters), name='se_reshape')(se)

    return tfkl.Multiply(name='se_multiply')([concat, se])


bottleneck = bottleneck_module(d4, 1024)
bottleneck = tfkl.SpatialDropout2D(0.4, name='bottleneck_dropout')(bottleneck)


# Upsampling path with skip connections
def upsampling_block(input_tensor, skip_connection, filters, name):
    up = tfkl.Conv2DTranspose(filters, kernel_size=2, strides=2, padding="same", name=f"{name}_upconv")(input_tensor)
    skip_connection = InstanceNormalization(name=f"{name}_skip_norm")(skip_connection)
    skip_connection = attention_gate(skip_connection, up, filters)
    skip_connection = attention_gate(skip_connection, up, filters)
    skip_connection = tfkl.Dropout(0.2)(skip_connection)  # Dropout sulle connessioni skip
    concat = tfkl.Concatenate(name=f"{name}_concat")([up, skip_connection])
    x = res_unet_block(concat, filters, name=f"{name}_unet")
    # Extra convolutional layer
    x = tfkl.Conv2D(filters, (3, 3), padding="same", kernel_initializer="he_normal", name=f"{name}_extra_conv")(x)
    x = tfkl.BatchNormalization(name=f"{name}_extra_bn")(x)
    x = tfkl.Activation("relu", name=f"{name}_extra_act")(x)
    return tfkl.SpatialDropout2D(0.2, name=f"{name}_dropout")(x)


# Upsampling path
u4 = upsampling_block(bottleneck, down_block_4, 512, name='up_block4')
u3 = upsampling_block(u4, down_block_3, 256, name='up_block3')
u2 = upsampling_block(u3, down_block_2, 128, name='up_block2')
u1 = upsampling_block(u2, down_block_1, 64, name='up_block1')
#u0 = upsampling_block(u1, down_block_1, 64, name='up_block0')



# Output layer with softmax activation for multi-class segmentation
output_layer = tfkl.Conv2D(num_classes, kernel_size=1, padding='same', activation="softmax", name='output_layer')(u1)

# Define the model
model = tfk.Model(inputs=input_layer, outputs=output_layer)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False, name='mean_iou')


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-6),
    loss=combined_loss_no_background,
    metrics=[mean_iou]  # Mantieni la tua metrica IoU per il monitoraggio
)


# Print the model summary
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19            │ (None, 64, 128, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_conv1    │ (None, 64, 128, 64)    │            640 │ input_layer_19[0][0]   │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_bn1      │ (None, 64, 128, 64)    │            256 │ down_block1_unet_conv… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_act1     │ (None, 64, 128, 64)    │              0 │ down_block1_unet_bn1[… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_conv2    │ (None, 64, 128, 64)    │         36,928 │ down_block1_unet_act1… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_bn2      │ (None, 64, 128, 64)    │            256 │ down_block1_unet_conv… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_shortcut │ (None, 64, 128, 64)    │            128 │ input_layer_19[0][0]   │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_add      │ (None, 64, 128, 64)    │              0 │ down_block1_unet_bn2[… │
│ (Add)                     │                        │                │ down_block1_unet_shor… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_unet_act2     │ (None, 64, 128, 64)    │              0 │ down_block1_unet_add[… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_extra_conv    │ (None, 64, 128, 64)    │         36,928 │ down_block1_unet_act2… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_extra_bn      │ (None, 64, 128, 64)    │            256 │ down_block1_extra_con… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_extra_act     │ (None, 64, 128, 64)    │              0 │ down_block1_extra_bn[… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ down_block1_pool          │ (None, 32, 64, 64)     │              0 │ down_block1_extra_act… │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 51,261,901 (195.55 MB)

 Trainable params: 51,250,381 (195.50 MB)

 Non-trainable params: 11,520 (45.00 KB)

## 🛠️ Train and Save the Model

In [111]:
history = model.fit(
    combined_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    batch_size=batch_size,
    callbacks=callbacks
).history

final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - loss: 0.7821 - mean_iou: 0.5713 - val_loss: 0.7813 - val_mean_iou: 0.6428 - learning_rate: 3.1250e-05
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 84s 774ms/step - loss: 0.7821 - mean_iou: 0.5100 - val_loss: 0.7814 - val_mean_iou: 0.6430 - learning_rate: 3.1250e-05
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 84s 774ms/step - loss: 0.7869 - mean_iou: 0.5459 - val_loss: 0.7815 - val_mean_iou: 0.6423 - learning_rate: 3.1250e-05
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 84s 773ms/step - loss: 0.7824 - mean_iou: 0.5608 - val_loss: 0.7816 - val_mean_iou: 0.6430 - learning_rate: 3.1250e-05
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 84s 776ms/step - loss: 0.7828 - mean_iou: 0.5631 - val_loss: 0.7814 - val_mean_iou: 0.6431 - learning_rate: 3.1250e-05
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 84s 774ms/step - loss: 0.7835 - mean_iou: 0.5202 - val_loss: 0.7816 - val_mean_iou: 0.6430 - learning_rate: 3.1250e-05
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 8

In [104]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

print(f"Model saved as {model_filename}")

Model saved as model_241212_210022.keras


## 📊 Test the model

To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set.



In [107]:

model = tfk.models.load_model(
    model_filename,
    custom_objects={
        'jaccard_loss_no_background': jaccard_loss_no_background,
        'combined_loss_no_background': combined_loss_no_background,
        'res_unet_block': res_unet_block,
        'InstanceNormalization': InstanceNormalization,
        'attention_gate': attention_gate
        'adaptive_dropout': adaptive_dropout,
        'AdaptiveDropout': AdaptiveDropout
        'dice_loss_no_background': dice_loss_no_background,
        'focal_loss_no_background': focal_loss_no_background
        'mean_iou': mean_iou
    }
)
print(f"Model loaded from {model_filename}")



Model loaded from model_241212_210022.keras


In [117]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 43s 136ms/step
Predictions shape: (10022, 64, 128)


## 💾 Save the predictions

In [118]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [119]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>